### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

mkdir: cannot create directory ‘/root/airflow/dags’: File exists


In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректоной работы веб приложения
# в среде Google Colab

!pip install pyngrok

!ngrok authtoken '' # найти его можно https://dashboard.ngrok.com/get-started/setup 

In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/endpoints/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

In [ ]:
## Вам необходимо воспользоваться макросами чтобы airflow корректно обрабатывал задачу.
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime


def my_func(hello, date, **context):
  print(hello)
  print(date)
  print(context['task'])


with DAG('dag', schedule_interval='@daily', 
          start_date= datetime(2021,1,1),
          end_date=datetime(2021,1,10)) as dag:

  python_task    = PythonOperator(
    task_id='python_task', 
    python_callable=my_func,
    op_kwargs= {
      'hello': 'Hello World',
      'date':  '{{ execution_date }}'
      }
    )